In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import sklearn.model_selection as model_selection
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import SGD
from numpy import newaxis
import keras
from keras import backend as K
from sklearn.metrics import classification_report
import tensorflow as tf
import io
from tensorflow.keras import layers
from keras.layers import GlobalAveragePooling1D, Conv1D, Conv2D

Using TensorFlow backend.


In [3]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [4]:
import nltk
import os
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vishaal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vishaal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vishaal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
os.chdir('../10_Data/20_Extracted Tweets/15_2018 Test')

In [6]:
os.chdir('../15_2018 Test')

FileNotFoundError: [WinError 2] The system cannot find the file specified: '../15_2018 Test'

In [383]:
df1 = pd.read_csv('Earthquake_TREC_2018_test.csv')
df2 = pd.read_csv('Floods_TREC_2018_test.csv')
#df3 = pd.read_csv('Earthquake_TREC_2018_train.csv')
#df4 = pd.read_csv('flood_TREC_2018_train.csv')

In [389]:
df4.head()

,Unnamed: 0,Unnamed: 0.1,ID,Tweet,Event,Retweet_Count,Follower_Count,Source,User_Created_at,Tweet_Created_at,User_Language,User_Screen_Name,User_Location,Event_Decrption,Categories,Priority
0,510,510,378011169883037697,"RT @dlfluegge: Crazy Flooding in Boulder, Colo...",floodColorado2013,10,376,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2009-07-26 18:15:13,2013-09-12 04:24:20,en,jewmike,Colorado,The 2013 Colorado floods was a natural disaste...,['MultimediaShare'],High
1,511,511,378020179214491649,Here's the #boulderflood video that's circulat...,floodColorado2013,3,211,"<a href=""http://twitter.com/download/iphone"" r...",2010-05-16 21:17:24,2013-09-12 05:00:08,en,KristinsDreams,"Denver, CO",The 2013 Colorado floods was a natural disaste...,['MultimediaShare'],High
2,512,512,378026101588496385,RT @passantino: Video: Severe flooding hits ne...,floodColorado2013,3,621,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2011-02-02 23:20:53,2013-09-12 05:23:40,en,mindfulnurse,"Boulder, Colorado",The 2013 Colorado floods was a natural disaste...,['MultimediaShare'],High
3,513,513,378029784204206080,"Crazy Flooding in Boulder, Colorado http://t.c...",floodColorado2013,0,232,"<a href=""http://www.apple.com"" rel=""nofollow"">...",2010-04-06 03:59:52,2013-09-12 05:38:18,en,JoshWorldPeace,NaN,The 2013 Colorado floods was a natural disaste...,['MultimediaShare'],High
4,514,514,378038458024865792,Thoughts and prayers out to those in Boulder w...,floodColorado2013,5,188,"<a href=""http://twitter.com/#!/download/ipad"" ...",2010-02-13 05:31:57,2013-09-12 06:12:46,en,jimatgoleaddog,"Lyme & Randolph, NH",The 2013 Colorado floods was a natural disaste...,['Sentiment'],Low


In [613]:
'''
    Combining all into one big data frame
'''
df = pd.DataFrame()
df['Tweet'] = pd.concat([df1['Tweet'] , df2['Tweet'], df3['Tweet'], df4['Tweet']])
df['Priority'] = pd.concat([df1['Priority'] , df2['Priority'], df3['Priority'], df4['Priority']])


In [457]:
df[df['Priority']=='Critical'].shape

(55, 2)

In [614]:
df_low = df[df['Priority'] == 'Low'].sample(55)
df_crit = df[df['Priority'] == 'Critical']
df_c = pd.concat([df_low, df_crit])
df_c = shuffle(df_c)

In [8]:
df_c = pd.read_csv('Ultimate_Dataframe_TREC_2018_Test.csv')
df_c = shuffle(df_c)

In [9]:
'''
Creating a categorical variable to keep label critical tweets as 1 and 0 otherwise
'''
def to_categorical(df_c):
    t = []
    for element in df_c['Priority']:
        if element =='Critical':
            t.append(1)
        else:
            t.append(0)
        
    t = np.array(t)
    df_c['Target'] = t

    df_c['Target'] = df_c['Target'].astype('category')
    t = df_c['Target']
    del df_c['Target']
    return (t)

t = to_categorical(df_c)

In [11]:
'''
    Creating a function to input lemmatized text to possibly another function that outputs the tfidf in a csv format.
    We could also simply use the output from this funtion in an tfidf format (no csv) and train a model.
'''
def preProcess(df):
    df['Tweet'] = df['Tweet'].astype('str')
    
    token_array = []
    for tweet in df['Tweet']:
        token_tweet = word_tokenize(tweet)
        token_array.append(token_tweet)
        
    stop_words=set(stopwords.words("english"))
    filtered_token_array=[]
    for tweet in token_array:
        filtered_tweet = []
        for word in tweet:
                if word not in stop_words:
                    filtered_tweet.append(word)
        filtered_token_array.append(filtered_tweet)
        
    lem = WordNetLemmatizer()
    stem = PorterStemmer()

    lemmatized_array=[]
    for tweet in filtered_token_array:
        lemmatized_tweet = []
        for word in tweet:
            lemmatized_tweet.append(lem.lemmatize(word,'v'))
        lemmatized_array.append(lemmatized_tweet)
    
    lemmatized_array_join = []
    for element in lemmatized_array:
        lemmatized_array_join.append(' '.join(element))
        
    return (lemmatized_array_join)


In [12]:
'''
  We tokenize the lemmatized tweets to then do word embeddings
'''
l = preProcess(df_c)

all_words = []

for tweet in l:
    tokenize_word = word_tokenize(tweet)
    for word in tokenize_word:
        all_words.append(word)
'''
    Getting the unique words out
'''
unique_words = set(all_words)

In [13]:
'''
    Getting embedded sentences
'''
vocab_length = len(unique_words)

embedded_sentences = [one_hot(tweet, vocab_length) for tweet in l]

In [619]:
'''
    Making the size of all embeddings equal to the longest one
'''
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(l, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

In [620]:
vocab_length

939

In [621]:
'''
    Model Paramters
'''
model = keras.Sequential()
model.add(Embedding(vocab_length, 16, input_length=padded_sentences.shape[1]))
model.add(GlobalAveragePooling1D())
model.add(Dense(8, activation = 'relu', input_shape = (padded_sentences.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))


model.summary()


Model: "sequential_109"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_103 (Embedding)    (None, 34, 16)            15024     
_________________________________________________________________
global_average_pooling1d_86  (None, 16)                0         
_________________________________________________________________
dense_180 (Dense)            (None, 8)                 136       
_________________________________________________________________
dropout_36 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_181 (Dense)            (None, 1)                 9         
Total params: 15,169
Trainable params: 15,169
Non-trainable params: 0
_________________________________________________________________


In [623]:
'''
    Compile and fit model
'''
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

history = model.fit(padded_sentences, t, batch_size=8, epochs=20, validation_split=0.4)

Train on 66 samples, validate on 44 samples
Epoch 1/20
66/66 [==============================] - 0s 4ms/step - loss: 0.6805 - precision_76: 0.6604 - recall_74: 0.9722 - val_loss: 0.6912 - val_precision_76: 0.4318 - val_recall_74: 1.0000
Epoch 2/20
66/66 [==============================] - 0s 287us/step - loss: 0.6740 - precision_76: 0.6471 - recall_74: 0.9167 - val_loss: 0.6909 - val_precision_76: 0.4318 - val_recall_74: 1.0000
Epoch 3/20
66/66 [==============================] - 0s 363us/step - loss: 0.6752 - precision_76: 0.6296 - recall_74: 0.9444 - val_loss: 0.6882 - val_precision_76: 0.4524 - val_recall_74: 1.0000
Epoch 4/20
66/66 [==============================] - 0s 393us/step - loss: 0.6705 - precision_76: 0.6939 - recall_74: 0.9444 - val_loss: 0.6873 - val_precision_76: 0.4737 - val_recall_74: 0.9474
Epoch 5/20
66/66 [==============================] - 0s 514us/step - loss: 0.6593 - precision_76: 0.8140 - recall_74: 0.9722 - val_loss: 0.6844 - val_precision_76: 0.6296 - val_recall

In [16]:
iterations = 1

for i in range (iterations):
   
    '''
        Sample requsite tweets to balance class
    '''
    #df_low = df[df['Priority'] == 'Low'].sample(55)
    #df_crit = df[df['Priority'] == 'Critical']
    #df_c = pd.concat([df_low, df_crit])
    #df_c = shuffle(df_c)
    '''
        Categorize label as 0 or 1 (1 comprises of the + class aka critical tweets)
    '''
    t = to_categorical(df_c)

    '''
      We tokenize the lemmatized tweets to then do word embeddings
    '''
    l = preProcess(df_c)
    all_words = []
    for tweet in l:
        tokenize_word = word_tokenize(tweet)
        for word in tokenize_word:
            all_words.append(word)
    '''
        Getting the unique words out
    '''
    unique_words = set(all_words)

    '''
        Getting embedded sentences
    '''
    vocab_length = len(unique_words)
    embedded_sentences = [one_hot(tweet, vocab_length) for tweet in l]

    '''
        Making the size of all embeddings equal to the longest one
    '''
    word_count = lambda sentence: len(word_tokenize(sentence))
    longest_sentence = max(l, key=word_count)
    length_long_sentence = len(word_tokenize(longest_sentence))
    padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

    '''
        Model Paramters
    '''
    model = keras.Sequential()
    model.add(Embedding(vocab_length, 16, input_length=padded_sentences.shape[1]))
    model.add(GlobalAveragePooling1D())
    #model.add(Dense(8, activation = 'relu', input_shape = (padded_sentences.shape[1],)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = 'sigmoid'))

    '''
        Compile and fit model
    '''
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=[keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC()])

    history = model.fit(padded_sentences, t, batch_size=1024, epochs=100, validation_split=0.3)

Train on 11105 samples, validate on 4760 samples
Epoch 1/100
11105/11105 [==============================] - 1s 103us/step - loss: 0.6947 - precision_3: 0.0081 - recall_3: 0.5055 - auc_3: 0.5114 - val_loss: 0.6717 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_auc_3: 0.4987
Epoch 2/100
11105/11105 [==============================] - 0s 37us/step - loss: 0.6576 - precision_3: 0.0074 - recall_3: 0.0220 - auc_3: 0.4694 - val_loss: 0.6407 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_auc_3: 0.5183
Epoch 3/100
11105/11105 [==============================] - 0s 31us/step - loss: 0.6277 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.5300 - val_loss: 0.6117 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_auc_3: 0.5675
Epoch 4/100
11105/11105 [==============================] - 0s 31us/step - loss: 0.5985 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.5245 - val_loss: 0.5825 - val_precision_3: 0.0000e+00 - val_recall_3: 0.000

Epoch 34/100
11105/11105 [==============================] - 0s 35us/step - loss: 0.0673 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.5726 - val_loss: 0.0549 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_auc_3: 0.6667
Epoch 35/100
11105/11105 [==============================] - 0s 32us/step - loss: 0.0632 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.5919 - val_loss: 0.0524 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_auc_3: 0.6840
Epoch 36/100
11105/11105 [==============================] - 0s 37us/step - loss: 0.0611 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.5706 - val_loss: 0.0504 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_auc_3: 0.6516
Epoch 37/100
11105/11105 [==============================] - 0s 33us/step - loss: 0.0586 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.5588 - val_loss: 0.0488 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_auc_3: 0.6652
Epoc

Epoch 67/100
11105/11105 [==============================] - 1s 48us/step - loss: 0.0480 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.6571 - val_loss: 0.0447 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_auc_3: 0.6926
Epoch 68/100
11105/11105 [==============================] - 1s 48us/step - loss: 0.0484 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.6429 - val_loss: 0.0447 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_auc_3: 0.6934
Epoch 69/100
11105/11105 [==============================] - 1s 48us/step - loss: 0.0469 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.6762 - val_loss: 0.0447 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_auc_3: 0.6940
Epoch 70/100
11105/11105 [==============================] - 1s 46us/step - loss: 0.0463 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.6960 - val_loss: 0.0446 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_auc_3: 0.6992
Epoc

Epoch 100/100
11105/11105 [==============================] - 0s 38us/step - loss: 0.0444 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - auc_3: 0.7409 - val_loss: 0.0435 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - val_auc_3: 0.7756


In [650]:
import statistics
precision = [0.9231, 0.5625, 0.4545, 1, 0.5455, 0.5556, 0.75, 1, 1, 0.4318]
recall = [1, 0.3, 0.4848, 1, 1, 1, 0.2273,1, 1, 0.9524]
AUC = [0.7821, 0.8427, 0.8946, 0.8002, 0.8383, 0.8453,0.8874, 0.8, 0.8062, 0.7847 ]
print('Mean Precision = ', statistics.mean(precision))
print('Stdev Precision = ', statistics.stdev(precision))
print('Mean Recall = ', statistics.mean(recall))
print('Stdev Recall = ', statistics.stdev(recall))
print('Mean ROC AUC = ', statistics.mean(AUC))
print('Stdev ROC AUC = ', statistics.stdev(AUC))

Mean Precision =  0.7223
Stdev Precision =  0.23877699033012187
Mean Recall =  0.79645
Stdev Recall =  0.3232536095871338
Mean ROC AUC =  0.82815
Stdev ROC AUC =  0.04027536467867174


In [508]:
y_pred = model.predict(history.validation_data[0])

array([[0.7725922 ],
       [0.6906314 ],
       [0.2784298 ],
       [0.7214121 ],
       [0.9677178 ],
       [0.7108212 ],
       [0.00933427],
       [0.05144485],
       [0.9994686 ],
       [0.9985454 ],
       [0.9921698 ],
       [0.9908632 ],
       [0.00318617],
       [0.9987317 ],
       [0.9897058 ],
       [0.24104315],
       [0.9969379 ],
       [0.20989063],
       [0.95295864],
       [0.9522515 ],
       [0.05235607],
       [0.74673504],
       [0.11396532],
       [0.99802965],
       [0.8493108 ],
       [0.04973009],
       [0.95636237],
       [0.39834592],
       [0.992837  ],
       [0.94512135],
       [0.4136941 ],
       [0.26527405],
       [0.02235318]], dtype=float32)

In [506]:
metrics.confusion_matrix(y_true, y_pred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets